<a href="https://colab.research.google.com/github/geijinchan/Better-RAG-with-Merge-Retriever-LOTR-/blob/main/Better%20RAG%20(LOTR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Important Dependencies

In [ ]:
!pip install chromadb langchain transformers sentence-transformers
!pip install -U langchain-community
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 935.5 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.1

## Importing necessary dependencies

In [ ]:
import chromadb, torch, os
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma

## Getting the embedding model

In [3]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embedding': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

NameError: name 'HuggingFaceBgeEmbeddings' is not defined

In [1]:
print(f"Embedding model loaded \n{hf}")

NameError: name 'hf' is not defined

In [2]:
## Data Preprocessing

loader_un_sdg = PyPDFLoader("UN SDG.pdf")
documents_un_sdg = loader_un_sdg.load()
text_splitter_un_sdg = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts_un_sdg = text_splitter_un_sdg.split_documents(documents_un_sdg)

NameError: name 'PyPDFLoader' is not defined

In [ ]:
## Data Preprocessing

loader_paris_agreement = PyPDFLoader("english_paris_agreement.pdf")
documents_paris_agreement = loader_paris_agreement.load()
text_splitter_paris_agreement = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
texts_paris_agreement = text_splitter_paris_agreement.split_documents(documents_paris_agreement)

## Create and Store Vectors

In [ ]:
un_sdg_store = Chroma.from_documents(texts_un_sdg, hf, collection_metadata={"hnsw:space": "cosine"}, persist_directory="store/un_sdg_chroma_cosine")
paris_agreement_store = Chroma.from_documents(texts_paris_agreement, hf, collection_metadata={"hnsw:space": "cosine"}, persist_directory="store/paris_chroma_cosine")

## Load the vector store

In [ ]:
load_un_store = Chroma(persist_directory = "store/un_sdg_chroma_cosine",embedding_function=hf)
load_paris_store = Chroma(persist_directory = "store/paris_chroma_cosine",embedding_function=hf)

## Init Mege Retriever and Perform Semantic Search

In [ ]:
retriever_un_sdg = load_un_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3})
retriever_paris_agreement = load_paris_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [ ]:
lotr = MergerRetriever(retrievers=[retriever_un_sdg, retriever_paris_agreement])
query = "What does the goal 16 talks about"
docs = lotr.get_relevant_documents(query)
docs